In [1]:
import sys
!python setup.py install 

running install
running bdist_egg
running egg_info
writing carmen.egg-info/PKG-INFO
writing dependency_links to carmen.egg-info/dependency_links.txt
writing requirements to carmen.egg-info/requires.txt
writing top-level names to carmen.egg-info/top_level.txt
reading manifest file 'carmen.egg-info/SOURCES.txt'
writing manifest file 'carmen.egg-info/SOURCES.txt'
installing library code to build/bdist.macosx-10.9-x86_64/egg
running install_lib
running build_py
copying carmen/data/locations.json -> build/lib/carmen/data
creating build/bdist.macosx-10.9-x86_64/egg
creating build/bdist.macosx-10.9-x86_64/egg/carmen
copying build/lib/carmen/__init__.py -> build/bdist.macosx-10.9-x86_64/egg/carmen
copying build/lib/carmen/names.py -> build/bdist.macosx-10.9-x86_64/egg/carmen
creating build/bdist.macosx-10.9-x86_64/egg/carmen/resolvers
copying build/lib/carmen/resolvers/place.py -> build/bdist.macosx-10.9-x86_64/egg/carmen/resolvers
copying build/lib/carmen/resolvers/profile.py -> build/bdist.m

In [2]:
import numpy as np
import carmen
resolver = carmen.get_resolver()
resolver.load_locations()

/Users/samantharothman/Desktop/carmen/carmen/resolvers/profile.py:42: UserWarning: Duplicate location name "amman  jordan"
  warnings.warn('Duplicate location name "%s"' % alias)
/Users/samantharothman/Desktop/carmen/carmen/resolvers/profile.py:42: UserWarning: Duplicate location name "amman jordan"
  warnings.warn('Duplicate location name "%s"' % alias)
/Users/samantharothman/Desktop/carmen/carmen/resolvers/profile.py:42: UserWarning: Duplicate location name "st joseph county united states"
  warnings.warn('Duplicate location name "%s"' % alias)
/Users/samantharothman/Desktop/carmen/carmen/resolvers/profile.py:42: UserWarning: Duplicate location name "st tammany parish united states"
  warnings.warn('Duplicate location name "%s"' % alias)


In [3]:
def flatten_tweets(tweets):
    """ Flattens out tweet dictionaries so relevant JSON is 
        in a top-level dictionary. """
    
    tweets_list = []
    # Iterate through each tweet
    for tweet_obj in tweets:
        tweet_obj = json.loads(tweet_obj)
        ''' User info'''
        # Store the user screen name in 'user-screen_name'
        tweet_obj['user-screen_name'] = tweet_obj['user']['screen_name']
        
        # Store the user location
        tweet_obj['user-location'] = tweet_obj['user']['location']
        
        # Store number of retweets
        tweet_obj['retweet_number'] = tweet_obj['retweet_count']
        
        # Store number of favorites
        tweet_obj['favorite_number'] = tweet_obj['favorite_count']
        
        # Store hashtags
        if len(tweet_obj['entities']['hashtags']) != 0:
            tweet_obj['hashtags'] = []
            for x in range(len(tweet_obj['entities']['hashtags'])):
                tweet_obj['hashtags'].append(tweet_obj['entities']['hashtags'][x]['text']) 
        if len(tweet_obj['entities']['hashtags']) == 0:
            tweet_obj['hashtags'] = 'None'
        
        # Store data
        tweet_obj['date'] = tweet_obj['created_at']
        
        #new location
        tweet_obj['location1'] = resolver.resolve_tweet(tweet_obj)
    
        ''' Text info'''
        # Check if this is a 140+ character tweet
        if 'extended_tweet' in tweet_obj:
            # Store the extended tweet text in 'extended_tweet-full_text'
            tweet_obj['extended_tweet-full_text'] = \
                                    tweet_obj['extended_tweet']['full_text']
        if 'quoted_status' in tweet_obj:
            # Store the retweet user screen name in 
            #'retweeted_status-user-screen_name'
            tweet_obj['quoted_status-user-screen_name'] = \
                            tweet_obj['quoted_status']['user']['screen_name']

            # Store the retweet text in 'retweeted_status-text'
            tweet_obj['quoted_status-text'] = \
                                            tweet_obj['quoted_status']['text']
    
            if 'extended_tweet' in tweet_obj['quoted_status']:
                # Store the extended retweet text in 
                #'retweeted_status-extended_tweet-full_text'
                tweet_obj['quoted_status-extended_tweet-full_text'] = \
                    tweet_obj['quoted_status']['extended_tweet']['full_text']
        
        ''' Place info'''
        if 'place' in tweet_obj:
            # Store the country code in 'place-country_code'
            try:
                tweet_obj['place-country'] = \
                                            tweet_obj['place']['country']
                
                tweet_obj['place-country_code'] = \
                                            tweet_obj['place']['country_code']
                
                tweet_obj['location-coordinates'] = \
                            tweet_obj['place']['bounding_box']['coordinates']
            except: pass
        
        tweets_list.append(tweet_obj)
        
    return tweets_list
    """"
        if 'retweeted_status' in tweet_obj:
            # Store the retweet user screen name in 
            # 'retweeted_status-user-screen_name'
            tweet_obj['retweeted_status-user-screen_name'] = \
                        tweet_obj['retweeted_status']['user']['screen_name']

            # Store the retweet text in 'retweeted_status-text'
            tweet_obj['retweeted_status-text'] = \
                                        tweet_obj['retweeted_status']
    
            if 'extended_tweet' in tweet_obj['retweeted_status']:
                # Store the extended retweet text in 
                #'retweeted_status-extended_tweet-full_text'
                tweet_obj['retweeted_status-extended_tweet-full_text'] = \
                tweet_obj['retweeted_status']['extended_tweet']['full_text']
    """

In [4]:
def select_text(tweets):
    ''' Assigns the main text to only one column depending
        on whether the tweet is a RT/quote or not'''
    
    tweets_list = []
    
    # Iterate through each tweet
    for tweet_obj in tweets:
        
        if 'retweeted' == True:
            tweet_obj['text'] = \
                        tweet_obj['retweeted_status']['text']
        
        elif 'retweeted_status-text' in tweet_obj:
            tweet_obj['text'] = tweet_obj['retweeted_status-text']
            
        elif 'extended_tweet-full_text' in tweet_obj:
                    tweet_obj['text'] = tweet_obj['retweeted_status-text']
                
        tweets_list.append(tweet_obj)
        
    return tweets_list

In [7]:
import json
f = open('/Users/samantharothman/Documents/tweets4.json')
tweets = f.readlines()

In [8]:
import pandas as pd

#half = len(tweets)//15
# flatten tweets
#tweets1 = flatten_tweets(tweets[(half+half): (half+half+half)])
tweets1 = flatten_tweets(tweets)
#test = flatten_tweets(tweets[0:20])

# select text
#tweets2 = select_text(tweets1)
#test2 = select_text(test)

columns = ['text', 'lang', 'user-location', 'place-country', 
           'place-country_code', 'location-coordinates', 
           'user-screen_name', 'retweet_number','hashtags','date', 'location1']
# Create a DataFrame from `tweets`
df_tweets = pd.DataFrame(tweets1, columns=columns)
df_tweets.head()
# replaces NaNs by Nones
#df_tweets.where(pd.notnull(df_tweets), None, inplace=True)

,text,lang,user-location,place-country,place-country_code,location-coordinates,user-screen_name,retweet_number,hashtags,date,location1
0,This zombie has arrived on the scene and she i...,en,Orions Belt,NaN,NaN,NaN,VeganBombshell,1,None,Sat Oct 31 23:59:53 +0000 2020,None
1,Performing here tonite! Zombie style! What mor...,en,Orions Belt,NaN,NaN,NaN,VeganBombshell,1,None,Sat Oct 31 18:59:14 +0000 2020,None
2,Just posted a photo https://t.co/EXXyYLMBzU,en,Orions Belt,NaN,NaN,NaN,VeganBombshell,0,None,Sat Oct 31 18:51:49 +0000 2020,None
3,RT @iammantra: Today’s Mantra:\nI AM manifesti...,en,Orions Belt,NaN,NaN,NaN,VeganBombshell,2031,None,Fri Oct 30 14:15:40 +0000 2020,None
4,"Be back soon! Bday, Booked gigs, Halloween pre...",en,Orions Belt,NaN,NaN,NaN,VeganBombshell,0,None,Thu Oct 29 00:18:41 +0000 2020,None


In [11]:
df_tweets.head()

In [9]:
none = type(None)

In [10]:
def location2(place):
    if type(place) != none:
        return place
    else:
        return 'no'
df_tweets['location2'] = df_tweets.apply(lambda row: location2(row['location1']),axis=1)

In [11]:
use = df_tweets[df_tweets['location2'] != 'no']

In [12]:
def city(place):
    if type(place) != none:
        if place[1].city != '':
            return place[1].city
        else:
            return 'no'
    else:
        return 'no'
use['city'] = use.apply(lambda row: city(row['location2']),axis=1)

/Users/samantharothman/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  if __name__ == '__main__':


In [13]:
def state(place):
    if type(place) != none:
        if place[1].state != '':
            return place[1].state
        else:
            return 'no'
    else:
        return 'no'
use['state'] = use.apply(lambda row: state(row['location2']),axis=1)

/Users/samantharothman/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  if __name__ == '__main__':


In [14]:
use[(use['user-location'] != '') & (use['state'] == 'no')]['user-location'].unique()

array(['NYC', 'New York, NY', 'Uganda', 'United States',
       'Los Angeles, CA', 'Argentina', 'Ryukyu Islands, Japan.',
       'Seybaplaya, Campeche, Mexico', 'UK', 'United Kingdom',
       'Australia', 'Twin Cities, MN', 'Australia ', 'West Coast, USA',
       'Hungary', 'Canada', 'Take Action ➡️', 'Midwest', 'France',
       'Sacramento, CA', 'Sacramento, Calif.', 'Rancho Cordova',
       'Sacramento, CA 95814', 'Sacramento, California',
       'Sacramento, CA.', 'Sacramento', 'California, USA',
       'West Sacramento, California', 'Elk Grove, CA', 'Davis, CA',
       'Rocklin, California', 'Bay Area'], dtype=object)

In [15]:
where = use[(use['user-location'] != '') & (use['city'] == 'no')]['user-location'].index
where

Int64Index([  1988,   2963,   2964,   2965,   2966,   2968,   2969,   2970,
              2971,   3025,
            ...
            319913, 319914, 319915, 319916, 319940, 320606, 320608, 324081,
            324099, 326655],
           dtype='int64', length=72727)

In [16]:
for x in where:
    place = use.loc[x]['user-location']
    if ',' in place:
        s = place.split(',')
        if s[1] != ' US':
            city = s[0]
            state = s[1]
            if use.loc[x, 'city'] == 'Orange County' or "San Gabriel Valley":
                use.loc[x,'city'] = 'Los Angeles'
                use.loc[x,'state'] = 'California'
            if use.loc[x,'city'] == 'Sonoma County':
                use.loc[x,'city'] = 'San Francisco'
                use.loc[x,'state'] = 'California'
            else:
                use.loc[x,'city'] = city
                use.loc[x,'state'] = state
        else:
            state = s[0]
            use.loc[x,'state'] = state 
    else:
        if place == 'DC via Nigeria':
            use.loc[x,'city'] = 'D.C.'
            use.loc[x,'state'] = 'D.C.'
        if place == 'City of Brotherly Love':
            use.loc[x,'city'] = 'Philadelphia'
            use.loc[x,'state'] = 'Pennsylvania'
        if place == 'SOUTH PHILADELPHIA—':
            use.loc[x,'city'] = 'Philadelphia'
            use.loc[x,'state'] = 'Pennsylvania'

/Users/samantharothman/opt/anaconda3/lib/python3.7/site-packages/pandas/core/indexing.py:494: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[item] = s


In [17]:
use[(use['user-location'] != '') & (use['city'] == 'no')]['user-location'].unique()

array(['Orions Belt', 'NYC', 'Utah', 'Uganda', 'United States', 'London',
       'Nottinghamshire', 'Argentina', 'Michigan', 'Florida', 'LOUISIANA',
       'UK', 'New Jersey', 'United Kingdom', 'California', 'Australia',
       'Australia ', 'Hungary', 'Canada', 'Take Action ➡️', 'Midwest',
       'France', 'New York', 'Northern California',
       'NY/NJ/Los Angeles/Sacramento', 'Rancho Cordova', 'Sacramento',
       'Bay Area'], dtype=object)

In [18]:
use

,text,lang,user-location,place-country,place-country_code,location-coordinates,user-screen_name,retweet_number,hashtags,date,location1,location2,city,state
1988,As soon as house arrest is over... to the tree...,en,Orions Belt,United States,US,"[[[-124.482003, 32.528832], [-114.131212, 32.5...",VeganBombshell,0,None,Sat Apr 11 16:51:46 +0000 2020,"(False, Location(country='United States', stat...","(False, Location(country='United States', stat...",no,California
2963,@vickie_ivy Yea that one. But as a photoshoppy...,en,Orions Belt,United States,US,"[[[-124.482003, 32.528832], [-114.131212, 32.5...",VeganBombshell,0,None,Mon Mar 23 17:04:06 +0000 2020,"(False, Location(country='United States', stat...","(False, Location(country='United States', stat...",no,California
2964,@DebHiers1 It is called way too much botox ☝️😆,en,Orions Belt,United States,US,"[[[-124.482003, 32.528832], [-114.131212, 32.5...",VeganBombshell,0,None,Mon Mar 23 17:00:49 +0000 2020,"(False, Location(country='United States', stat...","(False, Location(country='United States', stat...",no,California
2965,@DebHiers1 It was a joke 👍 and yea she’s off h...,en,Orions Belt,United States,US,"[[[-124.482003, 32.528832], [-114.131212, 32.5...",VeganBombshell,0,None,Mon Mar 23 17:00:21 +0000 2020,"(False, Location(country='United States', stat...","(False, Location(country='United States', stat...",no,California
2966,@Ocean4Q @bigsexy1253 Yup seen em. My phone is...,en,Orions Belt,United States,US,"[[[-124.482003, 32.528832], [-114.131212, 32.5...",VeganBombshell,0,None,Mon Mar 23 16:58:49 +0000 2020,"(False, Location(country='United States', stat...","(False, Location(country='United States', stat...",no,California
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
333410,RT @mmurraypolitics: With the traditional camp...,en,Sacramento,NaN,NaN,NaN,Capitol_Weekly,6,None,Fri Mar 13 15:53:44 +0000 2020,"(False, Location(country='United States', stat...","(False, Location(country='United States', stat...",Sacramento,California
333411,RT @NateSilver538: I slightly worry that some ...,en,Sacramento,NaN,NaN,NaN,Capitol_Weekly,412,None,Fri Mar 13 15:45:30 +0000 2020,"(False, Location(country='United States', stat...","(False, Location(country='United States', stat...",Sacramento,California
333412,RT @aewright: Was literally double and triple ...,en,Sacramento,NaN,NaN,NaN,Capitol_Weekly,2,None,Fri Mar 13 15:44:47 +0000 2020,"(False, Location(country='United States', stat...","(False, Location(country='United States', stat...",Sacramento,California
333413,RT @FlashReport: Dear @Dodgers I understand po...,en,Sacramento,NaN,NaN,NaN,Capitol_Weekly,2,None,Fri Mar 13 15:44:14 +0000 2020,"(False, Location(country='United States', stat...","(False, Location(country='United States', stat...",Sacramento,California


In [19]:
use.to_csv('tweets4-carmen.csv')  # i added this